Lib for proccessing.

In [ ]:
import numpy as np
import cv2
import math

Lib for UI (Using Tkinter to build an interface)

In [ ]:
python -m pip install tkinter

In [ ]:
import tkinter as tk
from tkinter import *
import tkinter.filedialog
from PIL import Image, ImageTk

Decicate an image (double it's size by add an empty pixel around each root pixel) 

In [ ]:
def decmication(img):
    col, rows, ch = img.shape
    dcol = col * 2
    drows = rows * 2

    matriximage = np.zeros((dcol, drows, 3))

    for column in range(dcol):
        for row in range(drows):
            if column % 2 == 0 and row % 2 == 0:
                x = img[int(column / 2), int(row / 2)]
                matriximage[column, row] += x

    return matriximage

In [ ]:
zero_array = [0, 0, 0]

k = 1

global dstPath

dstPath = r'C:\Users\ADMIN\Desktop\anh2022\filter_result.png'

Check 3 point in a rows (column) if they're empty pixels [0,0,0].

In [ ]:
def checkVerticalZero(col, row, img):
    return (img[col, row] == zero_array).all() and (img[col, row - 1] == zero_array).all() and (
            img[col, row + 1] == zero_array).all()


def checkHorizontalZero(col, row, img):
    return (img[col, row] == zero_array).all() and (img[col - 1, row] == zero_array).all() and (
            img[col + 1, row] == zero_array).all()


Define U-value by equation in document

In [ ]:
def calculateU(a, b, c, d):
    numerator = k * (c - d) ** 2 + 1
    denominator = k * ((a - b) ** 2 + (c - d) ** 2) + 2

    return numerator / denominator

Fill the empty pixels by vertical and horizontal

In [ ]:
def verticalMask(col, row, img):
    pixel = [0, 0, 0]

    a_pixel = img[col, row - 3]
    b_pixel = img[col, row - 1]
    c_pixel = img[col, row + 1]
    d_pixel = img[col, row + 3]

    u_red = calculateU(a_pixel[0], b_pixel[0], c_pixel[0], d_pixel[0])
    u_green = calculateU(a_pixel[1], b_pixel[1], c_pixel[1], d_pixel[1])
    u_blue = calculateU(a_pixel[2], b_pixel[2], c_pixel[2], d_pixel[2])

    pixel[0] += int(u_red * b_pixel[0] + (1 - u_red) * c_pixel[0])
    pixel[1] += int(u_green * b_pixel[1] + (1 - u_green) * c_pixel[1])
    pixel[2] += int(u_blue * b_pixel[2] + (1 - u_blue) * c_pixel[2])

    return pixel


def horizontalMask(col, row, img):
    pixel = [0, 0, 0]

    a_pixel = img[col - 3, row]
    b_pixel = img[col - 1, row]
    c_pixel = img[col + 1, row]
    d_pixel = img[col + 3, row]

    u_red = calculateU(a_pixel[0], b_pixel[0], c_pixel[0], d_pixel[0])
    u_green = calculateU(a_pixel[1], b_pixel[1], c_pixel[1], d_pixel[1])
    u_blue = calculateU(a_pixel[2], b_pixel[2], c_pixel[2], d_pixel[2])

    pixel[0] += int(u_red * b_pixel[0] + (1 - u_red) * c_pixel[0])
    pixel[1] += int(u_green * b_pixel[1] + (1 - u_green) * c_pixel[1])
    pixel[2] += int(u_blue * b_pixel[2] + (1 - u_blue) * c_pixel[2])

    return pixel

Filtering a whole image

In [ ]:
def process(img):
    col, rows, ch = img.shape
    result = np.zeros((col, rows, ch))

    for column in range(3, col - 3):
        for row in range(3, rows - 3):
            if (img[column, row] == zero_array).all():
                if checkVerticalZero(column, row, img) and not checkHorizontalZero(column, row, img):
                    result[column, row] += horizontalMask(column, row, img)
                elif (not checkVerticalZero(column, row, img)) and checkHorizontalZero(column, row, img):
                    result[column, row] += verticalMask(column, row, img)
            else:
                result[column, row] += img[column, row]
    for column in range(3, col - 3):
        for row in range(3, rows - 3):
            if checkVerticalZero(column, row, img) and checkHorizontalZero(column, row, img):
                result[column, row] += (result[column - 1, row - 1] + result[column - 1, row] + result[
                    column - 1, row + 1] + result[column, row - 1] + result[column, row + 1] + result[
                                            column + 1, row - 1] + result[column + 1, row] + result[
                                            column + 1, row + 1]) * (1 / 8)

    for column in range(col - 1):
        for row in range(rows - 1):
            result[column, row, 0] = math.floor(result[column, row, 0])
            result[column, row, 1] = math.floor(result[column, row, 1])
            result[column, row, 2] = math.floor(result[column, row, 2])
    return result

For User Interface (Upload file and start processing when click button)